# 🚚 Late Delivery Risk Prediction - Machine Learning Project

**Author:** *Ezzat Mohammed*  
**Date:** *April 20, 2025*

---

### 🔍 Project Objective
The goal of this project is to build a machine learning model that can **predict whether a customer order will be delivered late** or not, based on various transactional, logistical, and customer-related features.

---

### 🎯 Why This Matters
Late deliveries can:
- Negatively affect customer satisfaction
- Increase operational costs
- Harm a brand’s reputation

By predicting late deliveries in advance, companies can take proactive steps to **optimize logistics** and **improve delivery performance**.

---

### 🛠️ Workflow Overview
1. **Data Cleaning & Preprocessing**  
2. **Exploratory Data Analysis (EDA)**  
3. **Feature Engineering**  
4. **Model Building (Classification)**  
5. **Model Evaluation & Interpretation**  

---




## 📦 Dataset Overview

The dataset contains transactional and logistical data related to customer purchases, delivery timelines, and product details. Below is a description of the key features included in the dataset.

### 🔑 Feature Descriptions

| Feature | Description |
|--------|-------------|
| `Type` | Type of transaction made |
| `Days for shipping (real)` | Actual shipping days of the purchased product |
| `Days for shipment (scheduled)` | Scheduled delivery days of the purchased product |
| `Benefit per order` | Earnings per order placed |
| `Sales per customer` | Total sales made per customer |
| `Delivery Status` | Delivery status of orders: Advance shipping, etc. |
| `Late_delivery_risk` | Indicates if the delivery was late (Yes/No) |
| `Category Id` | Product category code |
| `Category Name` | Description of the product category |
| `Customer Segment` | Type of customer: Consumer, Corporate, etc. |
| `Order Item Discount Rate` | Order item discount percentage |
| `Order Item Product Price` | Price of the product before discount |
| `Order Item Quantity` | Number of products in the order |
| `Sales` | Sales value of the order |
| `Order Profit Per Order` | Profit made per order |
| `Shipping Mode` | Shipping method used (Standard, Express, etc.) |
| `Order Region`, `Order Country`, `Order State` | Delivery destination |
| `order date (DateOrders)` | Date the order was placed |
| `Shipping date (DateOrders)` | Date the order was shipped |

> ⚠️ Non-essential features such as `Customer Email`, `Product Image`, or `Customer Password` may be dropped during preprocessing.

---


> Let's dive in and start building a smart solution to tackle late delivery risks! 📊🤖

In [28]:
# imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

In [29]:
df = pd.read_csv("DataCoSupplyChainDataset.csv", encoding='ISO-8859-1')
df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer Email                 180519 non-null  object 
 12  Customer Fname                

# 1. 🧹 Data Cleaning & Preparation

- **Handle missing values:**

In [31]:
def check_data_quality(df):
    """
    Simplified data quality check for missing and duplicated values.

    Parameters:
    df (pd.DataFrame): Input DataFrame to analyze

    Returns:
    tuple: 
        - pd.DataFrame: Columns 'Column_Name', 'Missing_Values', 'Missing_Percentage'
        - int: Number of duplicate rows
    """
    # Count missing values for each column
    missing_counts = df.isnull().sum()

    # Convert to DataFrame for better presentation
    result = missing_counts.reset_index()
    result.columns = ['Column_Name', 'Missing_Values']
    
    # Add percentage of missing values
    result['Missing_Percentage'] = (result['Missing_Values'] / len(df)) * 100

    # Count duplicate rows
    duplicate_count = df.duplicated().sum()

    return result, duplicate_count


missing_values, duplicates = check_data_quality(df)
print(missing_values)
print(f"\nNumber of duplicate rows: {duplicates}")


                      Column_Name  Missing_Values  Missing_Percentage
0                            Type               0            0.000000
1        Days for shipping (real)               0            0.000000
2   Days for shipment (scheduled)               0            0.000000
3               Benefit per order               0            0.000000
4              Sales per customer               0            0.000000
5                 Delivery Status               0            0.000000
6              Late_delivery_risk               0            0.000000
7                     Category Id               0            0.000000
8                   Category Name               0            0.000000
9                   Customer City               0            0.000000
10               Customer Country               0            0.000000
11                 Customer Email               0            0.000000
12                 Customer Fname               0            0.000000
13                  

| Column Name          | Missing Values | Missing Percentage |
|----------------------|-----------------|--------------------|
| Customer Zipcode     | 3               | 0.001662           |
| Order Zipcode        | 155679          | 86.239676          |
| Product Description  | 180519          | 100.000000         |

- **Convert dates to datetime format:**

In [39]:
import pandas as pd
from collections import Counter
from pprint import pprint

def analyze_column(column, pretty=False):
    """
    Analyzes a pandas Series (column) and returns key statistics.
    
    Args:
        column (pd.Series): A single column from a DataFrame
        pretty (bool): If True, prints the results in a pretty format
        
    Returns:
        dict: Dictionary containing:
            - 'min': minimum value
            - 'max': maximum value
            - 'value_counts': frequency of each value (as dict)
            - 'unique_values': list of unique values
            - 'unique_count': number of unique values
            - 'null_count': number of missing values
    """
    if not isinstance(column, pd.Series):
        raise TypeError("Input must be a pandas Series")

    value_counts = dict(column.value_counts())

    results = {
        'Min Value': column.min(),
        'Max Value': column.max(),
        'Most Common Values': value_counts,
        'Unique Values': column.unique().tolist(),
        'Unique Count': column.nunique(),
        'Missing (Null) Values': column.isna().sum()
    }

    if pretty:
        print("\n📊 Column Analysis Summary")
        print("-" * 30)
        for key, value in results.items():
            if isinstance(value, dict):
                print(f"\n{key}:")
                for k, v in value.items():
                    print(f"  • {k}: {v}")
            elif isinstance(value, list):
                print(f"{key}: [{', '.join(map(str, value))}]")
            else:
                print(f"{key}: {value}")
        print("-" * 30)

    return results


In [46]:
result = analyze_column(df['Category Name'], pretty=True)
result


📊 Column Analysis Summary
------------------------------
Min Value: Accessories
Max Value: Women's Golf Clubs

Most Common Values:
  • Cleats: 24551
  • Men's Footwear: 22246
  • Women's Apparel: 21035
  • Indoor/Outdoor Games: 19298
  • Fishing: 17325
  • Water Sports: 15540
  • Camping & Hiking: 13729
  • Cardio Equipment: 12487
  • Shop By Sport: 10984
  • Electronics: 3156
  • Accessories: 1780
  • Golf Balls: 1475
  • Girls' Apparel: 1201
  • Golf Gloves: 1070
  • Trade-In: 974
  • Video Games: 838
  • Children's Clothing: 652
  • Women's Clothing: 650
  • Baseball & Softball: 632
  • Hockey: 614
  • Cameras : 592
  • Toys: 529
  • Golf Shoes: 524
  • Pet Supplies: 492
  • Garden: 484
  • Crafts: 484
  • DVDs: 483
  • Computers: 442
  • Golf Apparel: 441
  • Hunting & Shooting: 440
  • Music: 434
  • Consumer Electronics: 431
  • Boxing & MMA: 423
  • Books : 405
  • Kids' Golf Clubs: 384
  • Health and Beauty: 362
  • Sporting Goods: 357
  • Lacrosse: 343
  • Tennis & Racquet: 3

{'Min Value': 'Accessories',
 'Max Value': "Women's Golf Clubs",
 'Most Common Values': {'Cleats': 24551,
  "Men's Footwear": 22246,
  "Women's Apparel": 21035,
  'Indoor/Outdoor Games': 19298,
  'Fishing': 17325,
  'Water Sports': 15540,
  'Camping & Hiking': 13729,
  'Cardio Equipment': 12487,
  'Shop By Sport': 10984,
  'Electronics': 3156,
  'Accessories': 1780,
  'Golf Balls': 1475,
  "Girls' Apparel": 1201,
  'Golf Gloves': 1070,
  'Trade-In': 974,
  'Video Games': 838,
  "Children's Clothing": 652,
  "Women's Clothing": 650,
  'Baseball & Softball': 632,
  'Hockey': 614,
  'Cameras ': 592,
  'Toys': 529,
  'Golf Shoes': 524,
  'Pet Supplies': 492,
  'Garden': 484,
  'Crafts': 484,
  'DVDs': 483,
  'Computers': 442,
  'Golf Apparel': 441,
  'Hunting & Shooting': 440,
  'Music': 434,
  'Consumer Electronics': 431,
  'Boxing & MMA': 423,
  'Books ': 405,
  "Kids' Golf Clubs": 384,
  'Health and Beauty': 362,
  'Sporting Goods': 357,
  'Lacrosse': 343,
  'Tennis & Racquet': 328,
  '